In [51]:
import polars as pl
import requests as rq
import json

In [3]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [4]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [52]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [53]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

In [54]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

#### List of coins

In [55]:
def get_list_of_coins():

    response = get_response("/coins/list", use_demo, "", PUB_URL)

    return pl.DataFrame(response) 

In [56]:
df_coins = get_list_of_coins()

In [57]:
df_coins.filter(pl.col("symbol") == "btc")

id,symbol,name
str,str,str
"""batcat""","""btc""","""batcat"""
"""bitcoin""","""btc""","""Bitcoin"""
"""blackrocktradingcurrency""","""btc""","""BlackrockTradingCurrency"""
"""osmosis-allbtc""","""btc""","""Osmosis allBTC"""


### OHLC data

In [58]:
def get_ohlc_data(coin_id, target_curr, days, precision):

    ohlc_params = {
            "vs_currency": target_curr,
            "days": days,
            "precision": precision
    }

    ohlc_list_response = get_response(f"/coins/{coin_id}/ohlc",
                                      use_demo,
                                      ohlc_params,
                                      PUB_URL)

    df_ohlc = pl.DataFrame(ohlc_list_response, 
                           schema = ["timestamp", "open", "high", "low", "close"],
                           orient = "row")

    df_ohlc = df_ohlc.with_columns(
                  pl.from_epoch(pl.col("timestamp"),
                                time_unit = "ms").alias("timestamp")
         )

    return df_ohlc

In [59]:
df_ohlc = get_ohlc_data("bitcoin", "usd", "14", "2")

In [60]:
df_ohlc

timestamp,open,high,low,close
datetime[ms],f64,f64,f64,f64
2025-05-22 08:00:00,111456.2,111544.46,110664.19,110931.8
2025-05-22 12:00:00,110993.06,111074.77,110418.95,110838.77
2025-05-22 16:00:00,110896.85,111451.29,110835.13,111335.63
2025-05-22 20:00:00,111309.93,111814.16,111286.85,111350.89
2025-05-23 00:00:00,111245.07,111560.36,110878.32,111560.36
…,…,…,…,…
2025-06-04 12:00:00,105407.11,105910.49,105292.85,105294.57
2025-06-04 16:00:00,105281.39,105483.88,104648.35,105483.88
2025-06-04 20:00:00,105446.23,105465.2,104968.63,105059.01


### TODO 
- Verify that high and low columns have been assigned correctly
- Try other target currencies
- Export to CSV, XLSX